In [ ]:
#piplite presence will be shown with these statememnts
try:    
    import piplite
except ModuleNotFoundError:
    piplite=None
# Use these packages for download of data
if piplite:
    await piplite.install("mplfinance")
    await piplite.install("jinja2")
    # This is for the fetch in piplite
    import js 
    from pyodide.ffi import to_js
    from js import Object
else:
    import yfinance
#3 Imports of all the files
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy as sp

pd.options.display.max_columns = None

---

# 03 Simple moving average crossover 

## About
A simple moving average trading occurs whenever the price of the stock moves over or below the average price of the stock for a said period of days

## Usage
Click the $>>$ icon above to run this script
Or run each of the cells individually as a Jupyter Notebook

## What?
This script provides you with an ability to determine a simple change in the moving average of the stock. 
Note: *Feel free to move around for appropriate day count at the moving average*

---

## Grabbing data

***The data being provisioned is only for educational purposes***


*Note: You can use the free service limits usage to only 100 symbols per minute. Tampering with this system will result in discontinuation of the service*

## Adding symbols

Please feel free to add any **NSE SYMBOL** to the below list

In [ ]:
symbols = "ADANIENT,ADANIPORTS,APOLLOHOSP,ASIANPAINT,AXISBANK,\
BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BHARTIARTL,BPCL,BRITANNIA,\
CIPLA,COALINDIA,\
DIVISLAB,DRREDDY,\
EICHERMOT,\
GRASIM,\
HCLTECH,HDFC,HDFCBANK,HDFCLIFE,HEROMOTOCO,HINDALCO,HINDUNILVR,\
ICICIBANK,INDUSINDBK,INFY,ITC,\
JSWSTEEL,\
KOTAKBANK,\
LT,\
M&M,MARUTI,\
NESTLEIND,NTPC,\
ONGC,\
POWERGRID,\
RELIANCE,\
SBIN,SBILIFE,SUNPHARMA,\
TATAMOTORS,TATASTEEL,TCS,TATACONSUM,TECHM,TITAN,\
ULTRACEMCO,UPL,\
WIPRO"

In [ ]:
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "1mo"
intervalOfData = "1d"
apiSymbol = []
for symbol in symbols.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2)
        stockPandas = stockPandas[::-1]
    stockPandasTot[apiSymbolIndivi] = stockPandas

## Simple moving average crossover

[![Moving Average crossover](https://th.bing.com/th/id/R.9de5e9b8cde12f94f618278cdd074c92?rik=yxyy%2bEHrN%2fP4ZQ&riu=http%3a%2f%2fnewtraderu.com%2fwp-content%2fuploads%2f2013%2f01%2fMovingAverage2.gif&ehk=76K1eC4aQMP%2fm4kgMZf420B2bk0Vpfzkhr2WvshtJPA%3d&risl=&pid=ImgRaw&r=0)](https://th.bing.com/th/id/R.9de5e9b8cde12f94f618278cdd074c92?rik=yxyy%2bEHrN%2fP4ZQ&riu=http%3a%2f%2fnewtraderu.com%2fwp-content%2fuploads%2f2013%2f01%2fMovingAverage2.gif&ehk=76K1eC4aQMP%2fm4kgMZf420B2bk0Vpfzkhr2WvshtJPA%3d&risl=&pid=ImgRaw&r=0)

One can choose the lengths of periods which they are interested in for crossovers
$$
L_1 = 5 \\
L_2 = 15 
$$

The moving average crossover signal is formed when 0th day smaller moving average is greater than the larger moving average
- No previous trend required
- $SMA_1 > SMA_2$
- dates weightages are till day 3 as follows
    - With weights as follows
    - $ day 0 : 15 $
    - $ day 1 : 10 $
    - $ day 2 : 5 $
    - $ day 3 : 2 $

In [ ]:
## Variables
# Your choice of moving average days
L_1 = 5
# Your choice of moving average days
# Please ensure L_2 is greater than L_1
L_2 = 15
# Number of days checked [Can we push this dates checked to different cell?]
datesChecked = len(stockPandas['close'])
# weights of each individual day (Shape in reshape function should be matched. )
weights = np.array([15, 10, 5, 2])
# Adding additional weights, will be helfull only while checking.
weights = np.append(weights, np.ones((datesChecked-4,1)))
signalBullDict = {}

In [ ]:
for symbol in stockPandasTot:
    selectData = stockPandasTot[symbol][::-1]
    #The rows have open, high, low, close, volume data
    SMA_1 = finta.TA.SMA(selectData,L_1)[::-1]
    SMA_2 = finta.TA.SMA(selectData,L_2)[::-1]
    diff = (SMA_1-SMA_2).fillna(0)
    sign_change = (diff < 0) & (diff.shift(1) > 0)
    signalBull = sign_change.astype(int).to_numpy()
    signalBull = weights*signalBull
    signalBullDict[symbol] = signalBull
# Uncomment the below line for debugging
# signalBullDict

---
### Displaying results 
In this section we shall see the various manners that we can display the results

The below dataframe shows how the bullish signal has appeared on the chart for the past 50 days

In [ ]:
signalBullPand = pd.DataFrame(signalBullDict).T

def color_mapper(row):
    # compute the sum of the row
    sum = row.sum()
    # if the sum is negative, return red
    if sum < 0:
        return ['color: white; background-color: rgb(120, 50, 20)'] * len(row)
    # if the sum is positive, map it to a shade of green based on its magnitude
    else:
        magnitude = sum*10
        r = 15
        g = magnitude
        b = 15
        return [f'color: white; background-color: rgb({r},{g},{b})'] * len(row)

def make_pretty(styler):
    styler.apply(color_mapper,axis=1)
    styler.format(precision=1)
    return styler

signalBullPand.style.pipe(make_pretty)

In [ ]:
for symbol in stockPandasTot:
    stockPandas = stockPandasTot[symbol]
    print(symbol)
    mpf.plot(stockPandas.iloc[::-1],type='candle',mav=(5,15),volume=True)